# render-metro-views

Display the views that the metro application is providing. 

### Before you begin,  verify that you've executed [build-metro-application](./build-metro-application.jupyter-py36.ipynb) notebook.
The build-metro-application notebook composes and submits the metro application that recieves status from the edge, this notebook renders data from the metro application.

The metro application recieves messages on two topics from the edge. The 'Classification' messages provide statistics on the scoring on the edge, these messages are aggreagted into 25 elements. The 'Uncertain' message contains images that have a 
lower than acceptable confidence rating that require deeper analysis.

This notebook renders the data processed by the metro node. 

In [ ]:
%matplotlib inline
%gui asyncio
import urllib3, time, threading, base64, sys, IPython, io
#from IPython import display  ## DO NOT use interferes with display()
import ipywidgets as widgets
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from IPython.core.debugger import set_trace
## 
from icpd_core import icpd_util
from streamsx import rest
from streamsx.topology import context
from streamsx.rest_primitives import Instance
if 'scripts' not in sys.path:
    sys.path.insert(0, 'scripts')
import digit_support
import metrorender
urllib3.disable_warnings()
# Juypter 5 has the async handling fixes..
!jupyter --version

In [ ]:
# Cell to grab Streams instance config object and REST reference
STREAMS_INSTANCE_NAME = "<INSERT-STREAMS-INSTANCE-NAME-HERE>"
streams_cfg=icpd_util.get_service_instance_details(name=STREAMS_INSTANCE_NAME)
streams_cfg[context.ConfigParams.SSL_VERIFY] = False
instance = Instance.of_service(streams_cfg)

# Verify that Streams is active and Views being served up.

In [ ]:
# Verify that the job is healthy/up before proceding..

urllib3.disable_warnings()
# list the active jobs 
digit_support.list_jobs(instance, cancel=False)

In [ ]:
# List the active views - should see 3 here.

digit_support.display_views(instance, job_name="EdgeMetroSubscribe")

# Bring up the live Queues of Views
- ClassificationMetrics
- WindowUncertain

In [ ]:
# WindowUncertain - queue
output_WindowUncertain = widgets.Output()
display(output_WindowUncertain )
WindowUncertain_vtq = metrorender.view_to_queue(instance, "WindowUncertain", output_WindowUncertain)
WindowUncertain_vtq.start()  # start
#print("windowUncertain_thread\n\t alive:{}\n\t event:{}\n\t queue depth:{}".format(WindowUncertain_vtq.thread.is_alive(), WindowUncertain_vtq.event.is_set(), len(WindowUncertain_vtq.tuples)))
# WindowUncertain_vtq.event.clear()  # emergency kill

# UncertainPredictions - queue
output_UncertainPredictions = widgets.Output()
display(output_UncertainPredictions )
UncertainPredictions_vtq = metrorender.view_to_queue(instance, "UncertainPredictions", output_UncertainPredictions)
UncertainPredictions_vtq.start()  # start
#print("UncertainPredictions_thread\n\t alive:{}\n\t event:{}\n\t queue depth:{}".format(UncertainPredictions_vtq.thread.is_alive(), UncertainPredictions_vtq.event.is_set(), len(UncertainPredictions_vtq.tuples)))
# UncertainPredictions_vtq.event.clear()  # emergency kill

In [ ]:
%%script false --no-raise-error
# uncertain_vtq.event.clear()  # emergency kill - if the button does not work
time.sleep(2)
print("WindowUncertain_thread\n\t alive:{}\n\t event:{}\n\t queue depth:{}".format(
    WindowUncertain_vtq.thread.is_alive(), 
    WindowUncertain_vtq.event.is_set(), 
    len(WindowUncertain_vtq.tuples)))

print("UncertainPredictions_thread\n\t alive:{}\n\t event:{}\n\t queue depth:{}".format(UncertainPredictions_vtq.thread.is_alive(), UncertainPredictions_vtq.event.is_set(), len(UncertainPredictions_vtq.tuples)))
# UncertainPredictions_vtq.event.clear()  # emergency kill - if the button does not work

# Specify the cameras

- Discover the cameras that are avaliable by polling for X interations. Set the ACTIVE_CAMERAS to the cameras, use the checkboxes to change the cameras to observe.



In [ ]:
import json
chunks = WindowUncertain_vtq.tuples.copy()
ACTIVE_CAMERAS = set({})
for chunk in chunks:
    for tups in chunk:
        tup = json.loads(tups)
        key_list = tup['camera_metrics'].keys()
        ACTIVE_CAMERAS.add(list(key_list)[0])

#ACTIVE_CAMERAS = {'Camera-B4', 'Camera-A4', 'Camera-C2'}
ACTIVE_CAMERAS

# Window Uncertain image graphs

Display the throughput of each camera, the number of images processed and the % of images scored with certainty. 
Drops in certainty can be related to a myriad of issues including: model decay, hardware (camera) faults, environment (dust),...


In [ ]:
#%%script false --no-raise-error
idx = 1
while (len(WindowUncertain_vtq.tuples) < 5):
    print("priming{}".format(idx*"."),end="\n")
    idx += 1
    time.sleep(2)
print("primmed           ")
output_graphs = widgets.Output()
display(output_graphs)
synchronous_event = threading.Event()
synchronous = metrorender.deque_synchronous(WindowUncertain_vtq.tuples, count=5, debug=False)
rwu =  metrorender.RenderWindowUncertain(output_graphs, ACTIVE_CAMERAS)
try: 
    rwu.render(synchronous,synchronous_event)
except KeyboardInterrupt:
    print("Interrupt caught...")
    rwu.class_status_widget.value = "Interupt * Finished"
rwu.class_status_widget.value = "Rendering - Finished"
 

# Display sampled set uncertain images.

As images are scored images those that do not exceed the confidenc e level are returned
to the Metro edge. The edge would save this images for further anlaysis and be build 
a more robust model.

Below is a sampling of the uncertain images images, the images arrive at Metro node continuiously. 
The images are sampled from the Metro node and rendered in this notebook. 


In [ ]:
#%%script false --no-raise-error
# Un-threaded version
output_uncertain = widgets.Output()
display(output_uncertain)
rui = metrorender.RenderUncertainImages(output_uncertain)
rui.stop_button.description = "Use Interrupt"
rui.stop_button.tooltip = "Use Interrupt Kernel above"
active = True
try:
    while active:
        try:
            rui.display_view(UncertainPredictions_vtq.tuples.pop(), "live")
            time.sleep(2) # slow down - prevent widget overrunq
        except IndexError:
            time.sleep(3)
except KeyboardInterrupt:
    active = False
    rui.interrupt_stopped("Review displayed Images")

### Display images within a thread issues.
According to this [posting](https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html) 
you should pass the output widget into the thread. 
I'm doing it here, it works for the widgets other than images. Only the first images is displayed. When the thread is stopped and the review phase
is in effect it works.

Thus I've disabled it for now, to enable add a comment to the first line of the cell below. 

In [ ]:
%%script false --no-raise-error

output_uncertain = widgets.Output()
display(output_uncertain)

def thread_function(something, output):
    rui = metrorender.RenderUncertainImages()
    rui.next_button.disabled = True
    rui.previous_button.disabled = True
    rui.stop_button.disabled = False

    output.append_display_data(rui.dashboard)
    rui.active.set()
    while rui.active.is_set():
        try:
            rui.display_view(UncertainPredictions_vtq.tuples.pop(), "live")
            time.sleep(2) # slow down - prevent widget overrun
        except IndexError:
            time.sleep(3)

thread = threading.Thread(target=thread_function,args=("some ext", output_uncertain))
thread.start()


# Correction Station.

The current model is not perfect, overtime it encounters images that it cannot classify with confidence, these images are sent down the 'UncertainPrediction' view. In order to improve the model, the questionable images need to be assigned a value and added into the training data for the next round of model regeneration. The purpose of this
dashboard is to review the questionalble images and modify the prediction as necessary as well as expose other 
problem in the infrastructure that may have occured.

Not all the issues are due to model issues, those images need to be handled as well. Some images may be bad due to faulty hardware or camera misalighment.

The questional images and thier scores are displayed those that need to be correct, set the correct value. When done, the 'Further Processing' button will upload the images and corrections. The images tagged with updated digit values will be added to the next round of training data, the other images will be forwarded on to the appropriate party.

In [ ]:
#%%script false --no-raise-error

while len(UncertainPredictions_vtq.tuples)< 20:
    time.sleep(3)
    print(" - waiting for events ...")
snapShot = UncertainPredictions_vtq.tuples.copy()
cd = metrorender.CorrectionDashboard()
cd.render_review(snapShot)